In [18]:
import pandas as pd
import numpy as np
from datetime import datetime
from math import sqrt, cos


In [19]:
from module.distance_calculator import DistanceCalculator

In [20]:
df = pd.read_excel('sample_rbh_data.xlsx', sheet_name='Sheet1')

In [21]:
df.head()

,jobId,jobAcceptedTime,calledMerchantTime,arrivedAtMerchantTime,mealPickedUpTime,arrivedAtCustLocationTime,foodDeliveredTime,riderInitialLatLong.lat,riderInitialLatLong.lng,Merchant.Lat,Merchant.Lng,Customer.lat,Customer.lng
0,S210712297069,2021-07-12T11:51:59Z,2021-07-12T11:53:32Z,2021-07-12T11:55:42Z,2021-07-12T12:18:58Z,2021-07-12T12:23:48Z,2021-07-12T12:24:10Z,13.696167,100.372988,13.692471,100.378297,13.687792,100.3599
1,S210712176524,2021-07-12T04:54:43Z,2021-07-12T04:55:11Z,2021-07-12T05:08:57Z,2021-07-12T05:09:21Z,2021-07-12T05:26:02Z,2021-07-12T05:26:24Z,13.855861,100.505192,13.866007,100.497420,13.869973,100.455743
2,S210712207553,2021-07-12T06:19:40Z,2021-07-12T06:22:43Z,2021-07-12T06:39:45Z,2021-07-12T06:40:19Z,2021-07-12T06:55:35Z,2021-07-12T06:56:06Z,13.923222,100.641427,13.937330,100.625068,13.954999,100.611758
3,S210712202283,2021-07-12T06:03:43Z,2021-07-12T06:04:48Z,2021-07-12T06:12:18Z,2021-07-12T06:12:59Z,2021-07-12T06:26:30Z,2021-07-12T06:26:48Z,13.749551,100.650041,13.743053,100.624662,13.738476,100.692156
4,S210712235129,2021-07-12T08:12:10Z,2021-07-12T08:12:45Z,2021-07-12T08:17:26Z,2021-07-12T08:27:44Z,2021-07-12T08:50:51Z,2021-07-12T08:50:58Z,13.787922,100.780864,13.802771,100.782363,13.769007,100.722973


In [22]:
df.shape

(296, 13)

In [23]:
df["jobAcceptedTime"] = pd.to_datetime(df["jobAcceptedTime"], format="%Y-%m-%d %H:%M:%S")
df["calledMerchantTime"] = pd.to_datetime(df["calledMerchantTime"], format="%Y-%m-%d %H:%M:%S")
df["arrivedAtMerchantTime"] = pd.to_datetime(df["arrivedAtMerchantTime"], format="%Y-%m-%d %H:%M:%S")
df["mealPickedUpTime"] = pd.to_datetime(df["mealPickedUpTime"], format="%Y-%m-%d %H:%M:%S")
df["arrivedAtCustLocationTime"] = pd.to_datetime(df["arrivedAtCustLocationTime"], format="%Y-%m-%d %H:%M:%S")
df["foodDeliveredTime"] = pd.to_datetime(df["foodDeliveredTime"], format="%Y-%m-%d %H:%M:%S")

In [24]:
str_lat = [lat for lat in df["Customer.lat"].unique() if type(lat) == str]
str_lng = [lng for lng in df["Customer.lng"].unique() if type(lng) == str]
str_lat[:5]

[' ตำบลคูคต อำเภอลำลูกกา ปทุมธานี 12130 ประเทศไทย',
 ' Phasi Charoen',
 ' Min Buri',
 ' Thung Maha Mek',
 ' Pak Nam']

In [25]:
df = df.drop(df[df["Customer.lat"].isin(str_lat)].index)
df = df.drop(df[df["Customer.lng"].isin(str_lng)].index)

In [26]:
df["duration"] = (df["foodDeliveredTime"] - df["mealPickedUpTime"]).dt.total_seconds()//60

In [27]:
df.head()

,jobId,jobAcceptedTime,calledMerchantTime,arrivedAtMerchantTime,mealPickedUpTime,arrivedAtCustLocationTime,foodDeliveredTime,riderInitialLatLong.lat,riderInitialLatLong.lng,Merchant.Lat,Merchant.Lng,Customer.lat,Customer.lng,duration
0,S210712297069,2021-07-12 11:51:59+00:00,2021-07-12 11:53:32+00:00,2021-07-12 11:55:42+00:00,2021-07-12 12:18:58+00:00,2021-07-12 12:23:48+00:00,2021-07-12 12:24:10+00:00,13.696167,100.372988,13.692471,100.378297,13.687792,100.3599,5.0
1,S210712176524,2021-07-12 04:54:43+00:00,2021-07-12 04:55:11+00:00,2021-07-12 05:08:57+00:00,2021-07-12 05:09:21+00:00,2021-07-12 05:26:02+00:00,2021-07-12 05:26:24+00:00,13.855861,100.505192,13.866007,100.497420,13.869973,100.455743,17.0
2,S210712207553,2021-07-12 06:19:40+00:00,2021-07-12 06:22:43+00:00,2021-07-12 06:39:45+00:00,2021-07-12 06:40:19+00:00,2021-07-12 06:55:35+00:00,2021-07-12 06:56:06+00:00,13.923222,100.641427,13.937330,100.625068,13.954999,100.611758,15.0
3,S210712202283,2021-07-12 06:03:43+00:00,2021-07-12 06:04:48+00:00,2021-07-12 06:12:18+00:00,2021-07-12 06:12:59+00:00,2021-07-12 06:26:30+00:00,2021-07-12 06:26:48+00:00,13.749551,100.650041,13.743053,100.624662,13.738476,100.692156,13.0
4,S210712235129,2021-07-12 08:12:10+00:00,2021-07-12 08:12:45+00:00,2021-07-12 08:17:26+00:00,2021-07-12 08:27:44+00:00,2021-07-12 08:50:51+00:00,2021-07-12 08:50:58+00:00,13.787922,100.780864,13.802771,100.782363,13.769007,100.722973,23.0


In [28]:
def extract_datetime(df, datetime_col):
    df[datetime_col + ".dayOfWeek"] = df[datetime_col].dt.dayofweek
    # df[datetime_col + ".year"] = df[datetime_col].dt.year
    # df[datetime_col + ".month"] = df[datetime_col].dt.month
    # df[datetime_col + ".day"] = df[datetime_col].dt.day
    # df[datetime_col + ".hour"] = df[datetime_col].dt.hour
    # df[datetime_col + ".minute"] = df[datetime_col].dt.minute
    # df[datetime_col + ".second"] = df[datetime_col].dt.second
    df.drop(columns=[datetime_col], inplace=True)
    return df

In [29]:
# df = extract_datetime(df, "jobAcceptedTime")
# df = extract_datetime(df, "calledMerchantTime")
# df = extract_datetime(df, "arrivedAtMerchantTime")
# df = extract_datetime(df, "mealPickedUpTime")
# df = extract_datetime(df, "arrivedAtCustLocationTime")
# df = extract_datetime(df, "foodDeliveredTime")

In [30]:
df.head()

,jobId,jobAcceptedTime,calledMerchantTime,arrivedAtMerchantTime,mealPickedUpTime,arrivedAtCustLocationTime,foodDeliveredTime,riderInitialLatLong.lat,riderInitialLatLong.lng,Merchant.Lat,Merchant.Lng,Customer.lat,Customer.lng,duration
0,S210712297069,2021-07-12 11:51:59+00:00,2021-07-12 11:53:32+00:00,2021-07-12 11:55:42+00:00,2021-07-12 12:18:58+00:00,2021-07-12 12:23:48+00:00,2021-07-12 12:24:10+00:00,13.696167,100.372988,13.692471,100.378297,13.687792,100.3599,5.0
1,S210712176524,2021-07-12 04:54:43+00:00,2021-07-12 04:55:11+00:00,2021-07-12 05:08:57+00:00,2021-07-12 05:09:21+00:00,2021-07-12 05:26:02+00:00,2021-07-12 05:26:24+00:00,13.855861,100.505192,13.866007,100.497420,13.869973,100.455743,17.0
2,S210712207553,2021-07-12 06:19:40+00:00,2021-07-12 06:22:43+00:00,2021-07-12 06:39:45+00:00,2021-07-12 06:40:19+00:00,2021-07-12 06:55:35+00:00,2021-07-12 06:56:06+00:00,13.923222,100.641427,13.937330,100.625068,13.954999,100.611758,15.0
3,S210712202283,2021-07-12 06:03:43+00:00,2021-07-12 06:04:48+00:00,2021-07-12 06:12:18+00:00,2021-07-12 06:12:59+00:00,2021-07-12 06:26:30+00:00,2021-07-12 06:26:48+00:00,13.749551,100.650041,13.743053,100.624662,13.738476,100.692156,13.0
4,S210712235129,2021-07-12 08:12:10+00:00,2021-07-12 08:12:45+00:00,2021-07-12 08:17:26+00:00,2021-07-12 08:27:44+00:00,2021-07-12 08:50:51+00:00,2021-07-12 08:50:58+00:00,13.787922,100.780864,13.802771,100.782363,13.769007,100.722973,23.0


In [31]:
df.shape

(271, 14)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 271 entries, 0 to 295
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   jobId                      271 non-null    object             
 1   jobAcceptedTime            271 non-null    datetime64[ns, UTC]
 2   calledMerchantTime         271 non-null    datetime64[ns, UTC]
 3   arrivedAtMerchantTime      271 non-null    datetime64[ns, UTC]
 4   mealPickedUpTime           271 non-null    datetime64[ns, UTC]
 5   arrivedAtCustLocationTime  271 non-null    datetime64[ns, UTC]
 6   foodDeliveredTime          271 non-null    datetime64[ns, UTC]
 7   riderInitialLatLong.lat    271 non-null    float64            
 8   riderInitialLatLong.lng    271 non-null    float64            
 9   Merchant.Lat               271 non-null    float64            
 10  Merchant.Lng               271 non-null    float64            
 11  Custom

In [33]:
df.head()

,jobId,jobAcceptedTime,calledMerchantTime,arrivedAtMerchantTime,mealPickedUpTime,arrivedAtCustLocationTime,foodDeliveredTime,riderInitialLatLong.lat,riderInitialLatLong.lng,Merchant.Lat,Merchant.Lng,Customer.lat,Customer.lng,duration
0,S210712297069,2021-07-12 11:51:59+00:00,2021-07-12 11:53:32+00:00,2021-07-12 11:55:42+00:00,2021-07-12 12:18:58+00:00,2021-07-12 12:23:48+00:00,2021-07-12 12:24:10+00:00,13.696167,100.372988,13.692471,100.378297,13.687792,100.3599,5.0
1,S210712176524,2021-07-12 04:54:43+00:00,2021-07-12 04:55:11+00:00,2021-07-12 05:08:57+00:00,2021-07-12 05:09:21+00:00,2021-07-12 05:26:02+00:00,2021-07-12 05:26:24+00:00,13.855861,100.505192,13.866007,100.497420,13.869973,100.455743,17.0
2,S210712207553,2021-07-12 06:19:40+00:00,2021-07-12 06:22:43+00:00,2021-07-12 06:39:45+00:00,2021-07-12 06:40:19+00:00,2021-07-12 06:55:35+00:00,2021-07-12 06:56:06+00:00,13.923222,100.641427,13.937330,100.625068,13.954999,100.611758,15.0
3,S210712202283,2021-07-12 06:03:43+00:00,2021-07-12 06:04:48+00:00,2021-07-12 06:12:18+00:00,2021-07-12 06:12:59+00:00,2021-07-12 06:26:30+00:00,2021-07-12 06:26:48+00:00,13.749551,100.650041,13.743053,100.624662,13.738476,100.692156,13.0
4,S210712235129,2021-07-12 08:12:10+00:00,2021-07-12 08:12:45+00:00,2021-07-12 08:17:26+00:00,2021-07-12 08:27:44+00:00,2021-07-12 08:50:51+00:00,2021-07-12 08:50:58+00:00,13.787922,100.780864,13.802771,100.782363,13.769007,100.722973,23.0


In [42]:
def get_euc(coords_1, coords_2):
    R = 6371000
    conversion_const = 0.0174533

    c_1 = (coords_1[0]*conversion_const,coords_1[1]*conversion_const)
    c_2 = (coords_2[0]*conversion_const,coords_2[1]*conversion_const)


    delta_phi = abs(c_1[1]-c_2[1])

    theta = c_1[0]
    delta_theta = abs(c_1[0]-c_2[0])

    del_x = R*cos(theta)*delta_phi 
    del_y = R*delta_theta

    return sqrt(del_x**2+del_y**2)

In [43]:
df["eucDistanceFromRiderToCustomer"] = df.apply(lambda x: get_euc((x["Merchant.Lat"],x["Merchant.Lng"]),(x["Customer.lat"],x["Customer.lng"])), axis=1)

In [45]:
D = DistanceCalculator()

In [46]:
D.G.nodes()[5880824979]

{'y': 13.8349443, 'x': 100.5238375, 'street_count': 1}

In [47]:
D.G.nodes()[7903748995]

{'y': 13.8212094, 'x': 100.5076869, 'street_count': 3}

In [48]:
df["astarDistanceFromRiderToCustomer"] = df.apply(lambda x: D.shortestDistance((x["Merchant.Lat"],x["Merchant.Lng"]),(x["Customer.lat"],x["Customer.lng"])), axis=1)

4525703343 4527467267
5880824979 7903748995


NetworkXNoPath: Node 7903748995 not reachable from 5880824979

In [58]:
from sklearn.ensemble import RandomForestRegressor
reg = RandomForestRegressor(n_estimators=1000, max_depth=7, random_state=0)

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

In [59]:
X = df[["eucDistanceFromRiderToCustomer","riderInitialLatLong.lat","riderInitialLatLong.lng","Customer.lat","Customer.lng"]]
y = df["duration"]

In [60]:
X.head()

,eucDistanceFromRiderToCustomer,riderInitialLatLong.lat,riderInitialLatLong.lng,Customer.lat,Customer.lng
0,1693.089117,13.696167,100.372988,13.687792,100.3599
1,5564.311141,13.855861,100.505192,13.869973,100.455743
2,4768.541763,13.923222,100.641427,13.954999,100.611758
3,4712.532026,13.749551,100.650041,13.738476,100.692156
4,6596.003768,13.787922,100.780864,13.769007,100.722973


In [61]:
y.head()

0     5.0
1    17.0
2    15.0
3    13.0
4    23.0
Name: duration, dtype: float64

In [62]:
reg.fit(X,y)

RandomForestRegressor(max_depth=7, n_estimators=1000, random_state=0)

In [63]:
cv_results = cross_validate(reg, X, y, cv=5, scoring=('neg_mean_absolute_error', 'neg_mean_squared_error', 'r2'), return_train_score=True)

In [64]:
pd.DataFrame(cv_results)

,fit_time,score_time,test_neg_mean_absolute_error,train_neg_mean_absolute_error,test_neg_mean_squared_error,train_neg_mean_squared_error,test_r2,train_r2
0,1.035934,0.057637,-3.433252,-2.031565,-18.276326,-6.869408,0.372578,0.887476
1,1.083310,0.051141,-4.044029,-1.941068,-33.000172,-6.093671,0.390774,0.889254
2,1.084532,0.050558,-4.022726,-1.936614,-37.560836,-5.766881,0.516711,0.883260
3,1.018981,0.050020,-2.922684,-2.018832,-14.884128,-6.516797,0.601830,0.890257
4,1.029506,0.057999,-4.383037,-1.846816,-29.975771,-5.912920,0.590291,0.882989


In [65]:
reg.score(X, y)

0.8752404802982888